# 00 - Setup y credenciales

Este notebook verifica tu acceso a la API de Talkwalker: token, créditos restantes y lista de proyectos. Puedes usar el **token demo** (`demo`) para probar sin credenciales propias.

## 1. Configuración

In [ ]:
# @title Credenciales { display-mode: "form" }
ACCESS_TOKEN = "demo"  # @param {type:"string"} (usa "demo" para pruebas o tu token propio)

BASE_URL = "https://api.talkwalker.com"
import requests
import json

def tw_get(endpoint, params=None):
    p = {"access_token": ACCESS_TOKEN}
    if params:
        p.update(params)
    r = requests.get(f"{BASE_URL}{endpoint}", params=p)
    r.raise_for_status()
    return r.json()

def show(data):
    print(json.dumps(data, indent=2, ensure_ascii=False))

print("✅ Configuración lista. Token:", ACCESS_TOKEN[:10] + "..." if len(ACCESS_TOKEN) > 10 else ACCESS_TOKEN)

## 2. Verificar token: listar proyectos

In [ ]:
# Lista de proyectos accesibles con tu token (no consume créditos)
info = tw_get("/api/v1/search/info")
if info.get("status_code") != "0":
    print("Error:", info.get("status_message"))
else:
    projects = info.get("result_accinfo", {}).get("projects", [])
    print(f"Proyectos accesibles: {len(projects)}")
    for p in projects[:10]:
        print(f"  - {p.get('id')}: {p.get('name')}")
    if len(projects) > 10:
        print(f"  ... y {len(projects) - 10} más.")

## 3. Créditos restantes (solo con token propio)

In [ ]:
# Con token demo este endpoint puede no devolver créditos; con token propio sí.
try:
    cred = tw_get("/api/v1/status/credits")
    if cred.get("status_code") == "0":
        ci = cred.get("result_creditinfo", {})
        print("Créditos mensuales restantes:", ci.get("remaining_credits_monthly"))
        print("Usados este mes:", ci.get("used_credits_monthly"))
        print("Total mensual:", ci.get("monthly_total"))
    else:
        print("Respuesta:", cred.get("status_message"))
except Exception as e:
    print("Nota: con token demo puede no estar disponible.", e)

## 4. Resumen: rate limits y costes

- **Search API**: 60 llamadas/min (proyecto), 1 crédito/resultado + mín. 10 créditos/llamada.
- **Histogram API**: 30 llamadas/min (proyecto), 10 créditos/llamada.
- **Stream API**: 1 crédito por resultado.
- **Status credits**: 10 llamadas/min.